# Import and data type correction
> Importing libraries and correcting datatypes of the data columns


In [1]:
import os
import numpy as np
import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from datetime import datetime, timedelta
import pickle
import glob
from sklearn.impute import KNNImputer

!pip install reportlab pillow
from reportlab.platypus import SimpleDocTemplate, Paragraph, Image, Spacer, Table, TableStyle
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors

drive.mount('/content/gdrive/', force_remount=True)
os.chdir("/content/gdrive/Shareddrives/STA 221/data")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.2 MB/s eta 0:00:00
Mounted at /content/gdrive/


In [2]:
df = pd.read_csv("train.csv", low_memory=False)

In [3]:
# converting column types to appropriate type

# column id
df['id'] = df['id'].astype('category')
# column p_num
df['p_num'] = df['p_num'].astype('category')

# column time
df['time'] = pd.to_datetime(df['time'], errors='coerce')

# columns activity-*
for col in df.select_dtypes(include=["object"]).columns:
    df[col] = df[col].astype('category')

print(df.info())

<ipython-input-3-22d2651aec99>:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['time'] = pd.to_datetime(df['time'], errors='coerce')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177024 entries, 0 to 177023
Columns: 508 entries, id to bg+1:00
dtypes: category(74), datetime64[ns](1), float64(433)
memory usage: 604.6 MB
None


# Preprocessing
> To handle missing values without imputation and directly from the database, we followed this approach: First, we created an additional datetime column with datetime values for each entry in chronological order to have unique timestamps for each row. We inferred the correct timestamps based on the existing time-related columns (e.g., `bg-4:55`, `insulin-2:30`). For each row of each time-related column, we calculated the inferred timestamp using the time difference from the column name and stored the that cell's value in a dictionary `time_series_data` with the inferred timestamp as the key. Once the entire dataset is processed and `time_series_data` has been finalized, we then iterated over the dataset again, but now if the corresponding value at the cell is missing, we checked if the dictionary has a value at the inferred timestamp from that row and column, and updated the dataframe accordingly. _[In total, 2969 values were updates via this approach]_

## Adding datetime column

In [ ]:
# creating an additional datetime column that has date to have a chronological column with the data
# convert the `time` column to a time object
df['time'] = pd.to_datetime(df['time'], format='%H:%M:%S').dt.time

# initialize the starting date
start_date = datetime(2024, 1, 1)

# create a new `datetime` column to store the unique datetime
current_date = start_date
datetimes = []

# iterate over the rows to assign unique datetimes
for i, row in df.iterrows():
    if i > 0 and row['time'] < df.loc[i - 1, 'time']:
        # if the current time is less than the previous time, increment the date (daily reset)
        current_date += timedelta(days=1)

    # combine the current date with the time to create a unique datetime
    datetimes.append(datetime.combine(current_date, row['time']))

# add the new datetime column to the dataFrame
df['datetime'] = datetimes
df.to_csv('train_with_datetime.csv', index=False)

print(df)

## Updating missing values and saving df to `train_with_datetime.csv`

In [ ]:
# getting all time related columns
time_series_columns = [col for col in df.columns if col.startswith(('bg-', 'insulin-', 'carbs-', 'hr-', 'steps-', 'cals-'))]
# getting columns like bg, cals, carbs, hr, insulin, steps without their assiciated time
base_column_names = set([col.split('-')[0] for col in time_series_columns])
# base_column_names

In [ ]:
patients = df['p_num'].unique()
# patients

In [ ]:
# Function to save the intermediate state
def save_progress(data, processed_cols, save_path="dumps/time_series_data.pkl", log_path="dumps/processed_columns.log"):
    with open(save_path, "wb") as f:
        pickle.dump(data, f)
    with open(log_path, "w") as f:
        f.write("\n".join(processed_cols))
    print(f"Progress saved: {save_path} and {log_path}")

# Function to load the saved state
def load_progress(save_path="dumps/time_series_data.pkl", log_path="dumps/processed_columns.log"):
    if os.path.exists(save_path) and os.path.exists(log_path):
        with open(save_path, "rb") as f:
            data = pickle.load(f)
        with open(log_path, "r") as f:
            processed_cols = f.read().splitlines()
        print(f"Progress loaded: {save_path} and {log_path}")
        return data, processed_cols
    return None, []

# Check for saved progress
time_series_data, processed_columns = load_progress()

# If no progress exists, initialize variables
if time_series_data is None:
    # creating a dictionary for each column as key and a second dictionary as value
    # second dictionary has key as each unique patient and a `timeseries vs data` dictionary
    # eg {'steps': {'p01': {'timestamp1': data1, 'timestamp2': data2, ...}, 'p02': {'timestamp1': data1, 'timestamp2': data2, ...}, ...}, activity: {...}, ...}
    time_series_data = {name: {patient: {} for patient in patients} for name in base_column_names}
    processed_columns = []


Progress loaded: time_series_data.pkl and processed_columns.log


In [ ]:
# function to calculate the inferred timestamp
# returns the inferred timestamp by subtracting the time_diff (e.g., '4:55' from current_timestamp)
def get_inferred_timestamp(current_timestamp, time_diff):
    time_parts = time_diff.split(':')
    hours, minutes = int(time_parts[0]), int(time_parts[1])
    current_time = pd.to_datetime(current_timestamp)
    inferred_time = current_time - timedelta(hours=hours, minutes=minutes)
    return inferred_time

In [ ]:
# iterate over each raw time-related column
for col in time_series_columns:
    if col in processed_columns:
        print(f"Skipping already processed column: {col}")
        continue  # skip columns that were already processed

    # extract the base column name (e.g., 'bg', 'insulin', etc.) and time difference (e.g., '4:55')
    base_col, time_diff = col.split('-')[0], col.split('-')[1]
    print(f'Currently working on column: {col}')

    # process the data for the column
    for idx, row in df.iterrows():
        # extract the necessary values
        patient = row['p_num']  # patient ID
        timestamp = row['datetime']  # base timestamp for the row
        value = row[col]  # current column value

        # calculate the inferred timestamp
        inferred_timestamp = get_inferred_timestamp(timestamp, time_diff)

        # only process non-NaN values
        if inferred_timestamp not in time_series_data[base_col][patient]:
            # create an entry with None if it doesn't exist
            time_series_data[base_col][patient][inferred_timestamp] = None

        if not pd.isna(value):
            # if there's already a value, confirm consistency
            if time_series_data[base_col][patient][inferred_timestamp] is not None:
                # check if the new value matches the old value
                if time_series_data[base_col][patient][inferred_timestamp] != value:
                    # log or handle the inconsistency
                    print(f"Discrepancy found for {col}, Patient {patient}, Timestamp {inferred_timestamp}: "
                          f"Old Value: {time_series_data[base_col][patient][inferred_timestamp]}, "
                          f"New Value: {value}")
            # update the value (whether it's None or matches the new valid value)
            time_series_data[base_col][patient][inferred_timestamp] = value

    # mark the column as processed
    processed_columns.append(col)

    # save progress after each column
    save_progress(time_series_data, processed_columns)

In [ ]:
# loop over each column in time_series_data
for column in time_series_data:
    missing_count = 0  # to keep track of missing values in this column

    # loop over each patient in the current column
    for patient in time_series_data[column]:
        # loop over each timestamp for the patient in this column
        for timestamp, value in time_series_data[column][patient].items():
            if value is None:  # check if the value is missing (None)
                missing_count += 1

    # print the total number of missing values in the current column
    print(f"Total missing values in column '{column}': {missing_count}")


Total missing values in column 'bg': 56386
Total missing values in column 'carbs': 230481
Total missing values in column 'cals': 41818
Total missing values in column 'insulin': 9652
Total missing values in column 'hr': 60215
Total missing values in column 'steps': 119548


In [ ]:
# iterate over the `time_series_data` to update `df`
for col in time_series_columns:
    # extract the base column name (e.g., 'bg', 'insulin', etc.) and time difference (e.g., '4:55')
    base_col, time_diff = col.split('-')[0], col.split('-')[1]
    print(f'Currently working on column: {col}')

    # process the data for the column
    for idx, row in df.iterrows():
        # extract the necessary values
        patient = row['p_num']  # patient ID
        timestamp = row['datetime']  # base timestamp for the row
        value = row[col]  # current column value

        # calculate the inferred timestamp
        inferred_timestamp = get_inferred_timestamp(timestamp, time_diff)

        if inferred_timestamp in time_series_data[base_col][patient]:
            # check if the value in time_series_data is not None
            value_in_time_series = time_series_data[base_col][patient][inferred_timestamp]
            if value_in_time_series is not None:
                # update the corresponding cell in df
                if pd.isna(df.at[idx, col]):  # Update only if df cell is None/NaN
                    df.at[idx, col] = value_in_time_series
                    print(f"Updated df[{col}] for Patient: {patient}, Timestamp: {inferred_timestamp} with Value: {value_in_time_series}")


Currently working on column: bg-5:55
Updated df[bg-5:55] for Patient: p11, Timestamp: 2025-10-13 04:15:00 with Value: 11.1
Updated df[bg-5:55] for Patient: p11, Timestamp: 2025-10-13 04:20:00 with Value: 11.4
Updated df[bg-5:55] for Patient: p11, Timestamp: 2025-10-13 04:25:00 with Value: 11.5
Updated df[bg-5:55] for Patient: p11, Timestamp: 2025-10-13 04:30:00 with Value: 11.3
Updated df[bg-5:55] for Patient: p11, Timestamp: 2025-10-13 04:35:00 with Value: 11.2
Updated df[bg-5:55] for Patient: p11, Timestamp: 2025-10-13 04:40:00 with Value: 10.9
Updated df[bg-5:55] for Patient: p11, Timestamp: 2025-10-13 04:45:00 with Value: 10.8
Updated df[bg-5:55] for Patient: p11, Timestamp: 2025-10-13 04:50:00 with Value: 10.6
Updated df[bg-5:55] for Patient: p11, Timestamp: 2025-10-13 04:55:00 with Value: 10.5
Updated df[bg-5:55] for Patient: p11, Timestamp: 2025-10-13 05:00:00 with Value: 10.2
Updated df[bg-5:55] for Patient: p11, Timestamp: 2025-10-13 05:05:00 with Value: 9.8
Updated df[bg-5:55

In [ ]:
df.to_csv('train_with_datetime.csv', index=False)

# Filling activity columns
> We decided to fill the empty cells of the `activity-*` columns with `No_activity` based on the assumption that the patients were being tracked 24*7

In [4]:
df = pd.read_csv("train_with_datetime.csv", low_memory=False)
activity_columns = [col for col in df.columns if col.startswith('activity-')]

# fill missing values in activity columns with 'No_activity'
df[activity_columns] = df[activity_columns].fillna('No_activity')

df.to_csv('train_with_datetime.csv', index=False)
print("Activity columns filled with 'No_activity' and saved to 'train_with_datetime.csv'")


Activity columns filled with 'No_activity' and saved to 'train_with_datetime.csv'
